In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 17 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

13 February 2023 ; 18:09:10


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,25,0,8.8,1.5,3.4,0.440,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,1,1
1,Bobby Portis,PF,27,MIL,47,14,26.8,6.0,12.0,0.499,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1
2,Brook Lopez,C,34,MIL,55,55,30.4,5.5,10.9,0.503,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,0
3,George Hill,PG,36,MIL,35,0,19.1,1.7,3.8,0.447,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1
4,Giannis Antetokounmpo,PF,28,MIL,45,45,33.3,11.5,21.3,0.542,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Nikola Jokić,DEN,4.6,1.3,0.7,109.0,4.351020,1,0.184494
1,Victor Oladipo,MIA,1.6,1.7,0.2,109.0,3.784615,6,0.144179
2,Jaren Jackson Jr.,MEM,2.7,1.0,3.3,103.0,5.046154,2,0.142962
3,Darius Garland,CLE,-0.4,1.2,0.1,112.0,3.848980,4,0.118235
4,De'Anthony Melton,PHI,1.8,1.8,0.5,110.0,3.626923,3,0.115326


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)